In [3]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

def solve_navier_stokes(w0, nu, dt, t_final, N):
    """
    2D Navier–Stokes vorticity PDE (주기적 경계 조건) 를 pseudo‐spectral 방식으로 푸는 solver.
    
    PDE:
        ∂ₜw + u · ∇w = ν ∆w + f(x)
        u = (ψ_y, -ψ_x),  ∆ψ = -w,  with periodic boundary conditions.
        
    입력:
        w0: 초기 vorticity, shape (N, N)
        nu: 점성 계수
        dt: 시간 스텝
        t_final: 최종 시각 (예: 50)
        N: 격자 크기 (N×N)
        
    출력:
        snapshots: 매 정수 시간 t (0,1,2,...,t_final)마다 (t, w) 튜플의 리스트.
    """
    # 공간 좌표 생성 (0 ≤ x,y < 1)
    x = np.linspace(0, 1, N, endpoint=False)
    y = np.linspace(0, 1, N, endpoint=False)
    xv, yv = np.meshgrid(x, y, indexing='ij')
    
    # forcing function: f(x) = 0.1*( sin(2π(x+y)) + cos(2π(x+y)) )
    f = 0.1 * (np.sin(2*np.pi*(xv + yv)) + np.cos(2*np.pi*(xv + yv)))
    
    # FFT frequencies (주기적 도메인)
    kx = np.fft.fftfreq(N, d=1.0/N) * 2*np.pi
    ky = np.fft.fftfreq(N, d=1.0/N) * 2*np.pi
    kx, ky = np.meshgrid(kx, ky, indexing='ij')
    ksq = kx**2 + ky**2
    ksq[0,0] = 1.0  # division by zero 방지
    
    w = w0.copy()
    snapshots = []
    # 정수 시간마다 스냅샷 기록 (0,1,2,...,t_final)
    snapshot_times = np.arange(0, t_final + dt, 1.0)
    next_snapshot_index = 0
    nsteps = int(t_final / dt)
    
    for step in range(nsteps):
        t = step * dt
        if next_snapshot_index < len(snapshot_times) and t >= snapshot_times[next_snapshot_index]:
            snapshots.append((t, w.copy()))
            next_snapshot_index += 1
        
        # vorticity의 FFT
        w_hat = np.fft.fft2(w)
        # stream function: ψ_hat = -w_hat / ksq (단, 0번은 0으로)
        psi_hat = -w_hat / ksq
        psi_hat[0,0] = 0.0
        # 속도: u = ψ_y, v = -ψ_x (spectral differentiation)
        u = np.real(np.fft.ifft2(1j * ky * psi_hat))
        v = -np.real(np.fft.ifft2(1j * kx * psi_hat))
        # vorticity의 gradient
        w_x = np.real(np.fft.ifft2(1j * kx * w_hat))
        w_y = np.real(np.fft.ifft2(1j * ky * w_hat))
        nonlinear = u * w_x + v * w_y
        lap_w = np.real(np.fft.ifft2(-ksq * w_hat))
        
        # Euler time stepping
        w = w + dt * (-nonlinear + nu * lap_w + f)
    
    return snapshots

# -----------------------------
# 애니메이션 생성 및 .gif 저장
# -----------------------------
def create_flow_animation(w0, nu, dt, t_final, N, t_start, save_path='flow_animation.gif'):
    """
    주어진 초기 조건으로 Navier–Stokes PDE를 풀어 t = t_start부터 t_final까지의 vorticity 스냅샷을
    애니메이션으로 생성하고 .gif 파일로 저장합니다.
    
    입력:
        w0: 초기 vorticity, shape (N, N)
        nu: 점성 계수
        dt: 시간 스텝
        t_final: 최종 시각 (예: 50)
        N: 격자 크기
        t_start: 애니메이션에 포함할 시작 시간 (예: 10)
        save_path: 저장할 파일 이름
    """
    snapshots = solve_navier_stokes(w0, nu, dt, t_final, N)
    # t_start 이상인 스냅샷 필터링
    snapshots_filtered = [(t, w) for (t, w) in snapshots if t >= t_start]
    if len(snapshots_filtered) == 0:
        raise ValueError("t_start 이후의 스냅샷이 없습니다.")
    
    times = [t for (t, w) in snapshots_filtered]
    frames = [w for (t, w) in snapshots_filtered]
    
    fig, ax = plt.subplots()
    im = ax.imshow(frames[0], cmap="jet", origin="lower")
    ax.set_title("t = {:.1f}".format(times[0]))
    plt.colorbar(im, ax=ax)
    
    def update(frame):
        im.set_data(frames[frame])
        ax.set_title("t = {:.1f}".format(times[frame]))
        return [im]
    
    ani = animation.FuncAnimation(fig, update, frames=len(frames), interval=200, blit=True)
    
    # 애니메이션을 .gif 파일로 저장
    writer = animation.PillowWriter(fps=5)
    ani.save(save_path, writer=writer)
    print("Animation saved as", save_path)
    plt.close(fig)
    return ani

# -----------------------------
# 메인 실행: 초기 조건 생성 및 애니메이션 저장
# -----------------------------
if __name__ == '__main__':
    # PDE simulation parameters
    N = 64           # 공간 격자: 64x64
    dt = 0.005       # 시간 스텝
    t_final = 50     # 최종 시간: 50
    nu = 1e-3        # 점성 계수
    t_start = 10     # 애니메이션에 포함할 시작 시간
    
    # 초기 조건: 예를 들어, 가우시안 랜덤 필드
    w0 = np.random.randn(N, N)
    
    # 애니메이션 생성 및 .gif 저장
    ani = create_flow_animation(w0, nu, dt, t_final, N, t_start, save_path='navier_stokes_flow.gif')


Animation saved as navier_stokes_flow.gif
